In [33]:
import pandas as pd

In [34]:
df1 = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

In [35]:
df1

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21,4/12/21,4/13/21,4/14/21
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,56717,56779,56873,56943,57019,57144,57160,57242,57364,57492
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,126795,126936,127192,127509,127795,128155,128393,128518,128752,128959
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,117739,117879,118004,118116,118251,118378,118516,118645,118799,118975
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,12286,12328,12363,12409,12456,12497,12545,12581,12614,12641
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,22717,22885,23010,23108,23242,23331,23457,23549,23697,23841
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,NaN,Vietnam,14.058324,108.277199,0,2,2,2,2,2,...,2637,2648,2659,2668,2683,2692,2693,2705,2714,2733
270,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,253922,256461,259133,262017,264395,265897,268132,270856,272767,274690
271,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,4881,4975,5047,5133,5233,5276,5357,5446,5507,5582
272,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,89009,89071,89386,89592,89783,89918,90029,90064,90218,90389


In [36]:
df1.shape

(274, 453)

In [37]:
countries=df1["Country/Region"].count()
unique_countries=df1["Country/Region"].nunique(dropna = True)
print("There are ",countries ,"entries for the country column while unique no of countries is ",unique_countries)

There are  274 entries for the country column while unique no of countries is  192


This means there are repeated countries due to the seperate divisions for some countries inthe Province/State column like the US and Australia 

In [38]:
df1["Province/State"].nunique(dropna = True)

85

##### Drop the Province/State column, then group all rows by country.

In [39]:
df1=df1.drop('Province/State',axis=1)
df1.shape

(274, 452)

In [40]:
df1["Country/Region"].nunique(dropna = True)

192

In [42]:
df1=df1.groupby('Country/Region').sum()

In [43]:
df1.shape

(192, 451)

In [44]:
df1.head()

,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21,4/12/21,4/13/21,4/14/21
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,...,56717,56779,56873,56943,57019,57144,57160,57242,57364,57492
Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,...,126795,126936,127192,127509,127795,128155,128393,128518,128752,128959
Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,...,117739,117879,118004,118116,118251,118378,118516,118645,118799,118975
Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,...,12286,12328,12363,12409,12456,12497,12545,12581,12614,12641
Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,...,22717,22885,23010,23108,23242,23331,23457,23549,23697,23841


In [50]:
df1.columns

Index(['Lat', 'Long', '1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20',
       '1/27/20', '1/28/20', '1/29/20',
       ...
       '4/5/21', '4/6/21', '4/7/21', '4/8/21', '4/9/21', '4/10/21', '4/11/21',
       '4/12/21', '4/13/21', '4/14/21'],
      dtype='object', length=451)

In [ ]:
Add 